#  Setting Up a Ngnix Load Balancer

## Configure the Environment

In [1]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Set your FABRIC PROJECT ID
os.environ['FABRIC_PROJECT_ID']='6ce270de-788d-4e07-8bae-3206860a6387'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='cvankaya_0000027070'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_config/bastion-key-cvankaya'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import FABRIC API

In [2]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

## Step 4: Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated WAN Ethernet, as outlined in the [Create a Wide Area Ethernet (Layer 2) tutorial](../../fablib_api/create_l2network_wide_area.ipynb). More advanced topologies can be created, so long as the server node is routable by each client attempting to access the server's data.

In [3]:
SLICE_NAME = 'nginx-lb-slice'
SITE="UCSD"
NUM_SERVERS = 2

# We will use Ubuntu 20.04 for both nodes
IMAGE = 'default_ubuntu_20'
CORES = 1
RAM = 2
server_names = ["server_{}".format(i) for i in range(NUM_SERVERS)]
print(server_names)

['server_0', 'server_1']


In this example, we will set up an L2 network. As discuss above, a different network configuration, such as using the FABNet L3 network should work, so long as the clients and server can route to each other.

In [4]:
import datetime
from datetime import timedelta

try:
    #Create Slice
    slice = fablib.new_slice(name=SLICE_NAME)
    now = datetime.datetime.now(datetime.timezone.utc)
    end_date = (now + timedelta(days=6)).strftime("%Y-%m-%d %H:%M:%S %z")
    #slice.renew(end_date)
    
    #lb
    lb = {}
    lb_node = slice.add_node(name="ngnix_lb", site=SITE, cores=CORES, ram=RAM, image=IMAGE)
    lb["name"]="ngnix_lb"
    lb["node"]=lb_node
    lb["ifs"]={}
    #Create servers
    servers = {}

    for srv_name in server_names:
        server = slice.add_node(name=srv_name, site=SITE, cores=CORES, ram=RAM, image=IMAGE)
        server_iface = server.add_component(model='NIC_Basic', name="if_{}".format(srv_name)).get_interfaces()[0]
        servers[srv_name] = {"server":server, "interface": server_iface}
        lb_iface = lb_node.add_component(model='NIC_Basic', name="if_lb_{}".format(srv_name)).get_interfaces()[0]
        ser_net = slice.add_l2network(name="net_{}".format(srv_name), interfaces=[lb_iface, server_iface])
        lb["ifs"][srv_name] = {"if_name":"if_lb_{}".format(srv_name), "if": lb_iface, "net_name":"net_{}".format(srv_name), "net": ser_net}
        
    #client
    client = slice.add_node(name="client", site=SITE, image=IMAGE)
    client_iface = client.add_component(model='NIC_Basic', name="if_client").get_interfaces()[0]
    
    lb_iface = lb_node.add_component(model='NIC_Basic', name="if_lb_client").get_interfaces()[0]
    client_net = slice.add_l2network(name="net_client", interfaces=[lb_iface, client_iface])
    lb["ifs"]["client"] = {"if_name":"if_lb_client", "if": client_iface, "net_name":"net_client".format(srv_name), "net": client_net}
except Exception as e:
    print(f"Exception: {e}")

In [ ]:

#Submit Slice Request
slice.submit()


-----------  ------------------------------------
Slice Name   nginx-lb-slice
Slice ID     4a874099-50d0-4b74-880c-7204957a74da
Slice State  Configuring
Lease End    2022-12-02 07:21:08 +0000
-----------  ------------------------------------

Retry: 27, Time: 385 sec

ID                                    Name      Site    Host                          Cores    RAM    Disk  Image              Management IP    State    Error
------------------------------------  --------  ------  --------------------------  -------  -----  ------  -----------------  ---------------  -------  -------
f9883fca-bc01-40f9-9305-fc0851bfced9  ngnix_lb  UCSD    ucsd-w2.fabric-testbed.net        1      2      10  default_ubuntu_20  132.249.252.181  Active
9e2b3cb7-1a2b-4a51-af44-57437fb897ef  server_0  UCSD    ucsd-w2.fabric-testbed.net        1      2      10  default_ubuntu_20  132.249.252.162  Active
b7ed8956-ace0-46b7-8593-8cdecc71b028  server_1  UCSD    ucsd-w2.fabric-testbed.net        1      2      10  

## Print the Node Details

In [6]:
try:
    slice = fablib.get_slice(name=SLICE_NAME)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ---------------------------------------------------------------------------------------------------------------------------
ID                 b98de84a-e110-4863-9138-b2ec49af87ab
Name               ngnix_lb
Cores              1
RAM                2
Disk               10
Image              default_ubuntu_20
Image Type         qcow2
Host               ucsd-w2.fabric-testbed.net
Site               UCSD
Management IP      132.249.252.173
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -J cvankaya_0000027070@bastion-1.fabric-testbed.net ubuntu@132.249.252.173
-----------------  ---------------------------------------------------------------------------------------------------------------------------
-----------------  ---------------------------------------------------------------------------------------------------------------------------
ID                 17255a86-dcbf-48b6-bea7-d4be7b57285b
Name             

## Configure IP Addresses and Pick a Subnet

Create a subnet and list of available IP addresses. All objects are Python IP managment objects. You can use either IPv4 or IPv6 subents and addresses.

In [7]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("192.168.1.0/24")
    available_ips = list(subnet)[1:]
    print(available_ips)
except Exception as e:
    print(f"Exception: {e}")

[IPv4Address('192.168.1.1'), IPv4Address('192.168.1.2'), IPv4Address('192.168.1.3'), IPv4Address('192.168.1.4'), IPv4Address('192.168.1.5'), IPv4Address('192.168.1.6'), IPv4Address('192.168.1.7'), IPv4Address('192.168.1.8'), IPv4Address('192.168.1.9'), IPv4Address('192.168.1.10'), IPv4Address('192.168.1.11'), IPv4Address('192.168.1.12'), IPv4Address('192.168.1.13'), IPv4Address('192.168.1.14'), IPv4Address('192.168.1.15'), IPv4Address('192.168.1.16'), IPv4Address('192.168.1.17'), IPv4Address('192.168.1.18'), IPv4Address('192.168.1.19'), IPv4Address('192.168.1.20'), IPv4Address('192.168.1.21'), IPv4Address('192.168.1.22'), IPv4Address('192.168.1.23'), IPv4Address('192.168.1.24'), IPv4Address('192.168.1.25'), IPv4Address('192.168.1.26'), IPv4Address('192.168.1.27'), IPv4Address('192.168.1.28'), IPv4Address('192.168.1.29'), IPv4Address('192.168.1.30'), IPv4Address('192.168.1.31'), IPv4Address('192.168.1.32'), IPv4Address('192.168.1.33'), IPv4Address('192.168.1.34'), IPv4Address('192.168.1

## Configure the LB Interfaces

In [9]:
slice = fablib.get_slice(name=SLICE_NAME)
ip_map = {}
for name, iface in lb["ifs"].items():
    lb_node = slice.get_node("ngnix_lb")
    lb_iface = lb_node.get_interface(network_name=iface["net_name"]) 
    ip = available_ips.pop(0)
    ip_map[iface["if_name"]] = ip
    lb_iface.ip_addr_add(addr=ip, subnet=subnet)
    print("Configuring IP {} for {}".format(ip, name))
    stdout, stderr = lb_node.execute(f'ip addr show {lb_iface.get_os_interface()}')
    print (stdout)
    

"""  
server = slice.get_node(name=server_name)        
server_iface = server.get_interface(network_name=network_name) 
server_addr = available_ips.pop(0)
server_iface.ip_addr_add(addr=server_addr, subnet=subnet)

stdout, stderr = server.execute(f'ip addr show {server_iface.get_os_interface()}')
print (stdout)
"""

Configuring IP 192.168.1.1 for server_0
4: ens8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 1e:c2:9a:90:9d:8d brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.1/24 scope global ens8
       valid_lft forever preferred_lft forever
    inet6 fe80::1cc2:9aff:fe90:9d8d/64 scope link 
       valid_lft forever preferred_lft forever

Configuring IP 192.168.1.2 for server_1
6: ens10: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 26:9f:a3:3c:d0:15 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.2/24 scope global ens10
       valid_lft forever preferred_lft forever
    inet6 fe80::249f:a3ff:fe3c:d015/64 scope link 
       valid_lft forever preferred_lft forever

Configuring IP 192.168.1.3 for server_2
5: ens9: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 22:54:d5:53:48:1e brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.3/24 scope global ens9
       valid_

"  \nserver = slice.get_node(name=server_name)        \nserver_iface = server.get_interface(network_name=network_name) \nserver_addr = available_ips.pop(0)\nserver_iface.ip_addr_add(addr=server_addr, subnet=subnet)\n\nstdout, stderr = server.execute(f'ip addr show {server_iface.get_os_interface()}')\nprint (stdout)\n"

## Configure the server interfaces

In [10]:
print(server_names)
for name in server_names:
    ip = available_ips.pop(0)
    srv_node = slice.get_node(name)
    srv_iface = srv_node.get_interface(network_name="net_{}".format(name)) 
    print(ip)
    ip_map["if_{}".format(name)] = ip
    srv_iface.ip_addr_add(addr=ip, subnet=subnet)
    print("Configuring IP {} for {}".format(ip, name))
    stdout, stderr = srv_node.execute(f'ip addr show {srv_iface.get_os_interface()}')
    print (stdout)

['server_0', 'server_1', 'server_2']
192.168.1.5
Configuring IP 192.168.1.5 for server_0
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 2a:ce:d4:40:b1:cc brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.5/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::28ce:d4ff:fe40:b1cc/64 scope link 
       valid_lft forever preferred_lft forever

192.168.1.6
Configuring IP 192.168.1.6 for server_1
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 2a:da:63:3b:5f:35 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.6/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::28da:63ff:fe3b:5f35/64 scope link 
       valid_lft forever preferred_lft forever

192.168.1.7
Configuring IP 192.168.1.7 for server_2
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 2e:12:8f:2e:6f:47 brd ff

## Configure the Client's Interface

In [11]:
try:
    client = slice.get_node(name="client")        
    client_iface = client.get_interface(network_name="net_client") 
    client_addr = available_ips.pop(0)
    client_iface.ip_addr_add(addr=client_addr, subnet=subnet)
    ip_map["if_lb_client"] = ip
    stdout, stderr = client.execute(f'ip addr show {client_iface.get_os_interface()}')
    print (stdout)
    
except Exception as e:
    print(f"Exception: {e}")

3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 2e:a1:b4:ff:ee:4c brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.8/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::2ca1:b4ff:feff:ee4c/64 scope link 
       valid_lft forever preferred_lft forever



In [12]:
print(ip_map)

{'if_lb_server_0': IPv4Address('192.168.1.1'), 'if_lb_server_1': IPv4Address('192.168.1.2'), 'if_lb_server_2': IPv4Address('192.168.1.3'), 'if_lb_client': IPv4Address('192.168.1.7'), 'if_server_0': IPv4Address('192.168.1.5'), 'if_server_1': IPv4Address('192.168.1.6'), 'if_server_2': IPv4Address('192.168.1.7')}


## Install Nginx on the load balancer

In [13]:
try:
    print("Installing Ngnix...")
    lb_node = slice.get_node("ngnix_lb")
    stdout, stderr = lb_node.execute("sudo apt-get update && sudo apt install -y nginx")
    print(stdout)
    print("Finished installing Ngnix.")
    
except Exception as e:
    print(f"Exception: {e}")

Installing Ngnix...
Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1822 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [301 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 c-n-f Metadata [11.2 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1293 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [183 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 c-n-f Metadata [628 B]
Get:12 http://security.ubuntu.com/ubuntu

In [31]:
print(ip_map)

{'if_lb_server_0': IPv4Address('192.168.1.1'), 'if_lb_server_1': IPv4Address('192.168.1.2'), 'if_lb_server_2': IPv4Address('192.168.1.3'), 'if_lb_server_3': IPv4Address('192.168.1.4'), 'if_lb_server_4': IPv4Address('192.168.1.5'), 'if_lb_client': IPv4Address('192.168.1.16'), 'if_server_0': IPv4Address('192.168.1.12'), 'if_server_1': IPv4Address('192.168.1.13'), 'if_server_2': IPv4Address('192.168.1.14'), 'if_server_3': IPv4Address('192.168.1.15'), 'if_server_4': IPv4Address('192.168.1.16')}
